In [2]:
import cv2
import keras
import tensorflow as tf
import matplotlib.pyplot as plt

import os
import redis
import numpy as np
import redisConfig

from GA_audee import GA, ga_main
# from SourceLocalisation import SourceLocaliser
# from SourceLocalisation_F import SourceLocaliser_F

# start redis server and config the maximum memory available
redisConfig.run()

Configured


In [3]:
# model = keras.models.load_model('alexnet-cifar10_origin.h5')
# new_output = keras.layers.Softmax()(keras.layers.Dense(10)(model.layers[-2].output)) # without specifying activation 
# new_model = keras.models.Model(input=model.input, output=new_output)
# new_model.layers[-2].set_weights(model.layers[-1].get_weights())
# model = new_model

# model = keras.applications.ResNet50()
# new_output = keras.layers.Softmax()(keras.layers.Dense(1000)(model.layers[-2].output)) # without specifying activation 
# new_model = keras.models.Model(input=model.input, output=new_output)
# new_model.layers[-2].set_weights(model.layers[-1].get_weights())
# model = new_model

# model = keras.applications.MobileNetV2()
# new_output = keras.layers.Softmax()(keras.layers.Dense(1000)(model.layers[-2].output)) # without specifying activation 
# new_model = keras.models.Model(input=model.input, output=new_output)
# new_model.layers[-2].set_weights(model.layers[-1].get_weights())
# model = new_model

model = keras.applications.VGG16()
new_output = keras.layers.Softmax()(keras.layers.Dense(1000)(model.layers[-2].output)) # without specifying activation 
new_model = keras.models.Model(input=model.input, output=new_output)
new_model.layers[-2].set_weights(model.layers[-1].get_weights())
model = new_model

/data/yylaiai/anaconda3/envs/audee_test/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("so...)`


In [4]:
# load inputs
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# expand cifar10 images from 32x32 to 224x224
x_test_224 = np.array([cv2.resize(i, (224, 224)) for i in x_test])
print(x_test_224.shape)

(10000, 224, 224, 3)


In [5]:
mut_level = 'i+w'
model = model
x = x_test_224[0]/255
input_scale = 255 # image scale
init_noise = 0 # scale of Cauchy noise added in initialization
r1 = 0.5 # crossover rate
r2 = 0.3 # mutation rate
r3 = 0.01 # mutation scale (larger scale can trigger NaN inconsistency easier, smaller scale can increase layer distance easier)
m = 2 # top m chromosomes selected
n = 4 # population size
layer_idx = -2 # index of layer used in calculation of fitness values
db_flag = 3
maxIter = 3

r = redis.Redis(db=db_flag)
r.flushdb()

fit = ['inc', ['tensorflow', 'theano']]

In [6]:
ga = ga_main(fit, mut_level, model, x, input_scale, init_noise, r1, r2, r3, m, n, layer_idx, db_flag, 2)

Running at iteration 1:
Average fitness value: 5.8200210332870483e-05
Time taken: 183.34226512908936

Running at iteration 2:
Average fitness value: 0.00020592520013451576
Time taken: 87.51366806030273


Total time taken: 332.36450719833374


In [15]:
np.max(ga.fit_hist)

nan

In [17]:
np.max(np.nan_to_num(ga.fit_hist, -1))

5.820021033287048e-08

In [27]:
from GA_audee import GA, ga_main

# GA hyperparameters
mut_level = 'i+w'
init_noise = 0 # scale of Cauchy noise added in initialization
r1 = 0.5 # crossover rate
r2 = 0.3 # mutation rate
r3 = 0.01 # mutation scale
m = 2 # top m chromosomes selected
n = 4 # population size
layer_idx = -1 # index of layer used in calculation of fitness values
maxIter = 2


def ga_inc(backend_1, backend_2, model, x, input_scale, db_flag):
    fit = ['inc', [backend_1, backend_2]]
    ga = ga_main(fit, mut_level, model, x, input_scale, init_noise, r1, r2, r3, m, n, layer_idx, db_flag, maxIter)
    return np.max(np.nan_to_num(ga.fit_hist, -1))

In [32]:
m = keras.Sequential()
m.add(model.layers[1].from_config(model.layers[1].get_config()))

In [39]:
m.layers[0].get_config()

{'name': 'block1_conv1',
 'trainable': True,
 'dtype': 'float32',
 'filters': 64,
 'kernel_size': (3, 3),
 'strides': (1, 1),
 'padding': 'same',
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'activation': 'relu',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'scale': 1.0,
   'mode': 'fan_avg',
   'distribution': 'uniform',
   'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [33]:
ga_inc('tensorflow', 'theano', m, x, 255, 5)

ValueError: need at least one array to concatenate

In [34]:
ga.flattenPopulations([x])

array([[0.61960784, 0.43921569, 0.19215686, ..., 0.08235294, 0.2627451 ,
        0.43137255]])

In [38]:
m.layers[0]

[]

In [36]:
m.get_weights()

[]

In [ ]:
model_weights = self.model.get_weights()
chromosome = np.hstack([self.flattenPopulations([self.input]), self.flattenPopulations([model_weights])])
self.P = [chromosome]

for _ in range(n-1):
    noise = np.random.standard_cauchy(chromosome.shape) * init_noise
    self.P.append(chromosome + noise)